In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('movielens/users.dat',sep='::', header=None, names=unames)

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('movielens/ratings.dat',
                        sep='::', header=None, names=rnames)

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movielens/movies.dat',sep='::', header=None, names=mnames, encoding='cp1252')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('movielens/ratings.dat',sep='::', header=None, names=rnames)
                        

C:\Users\arunch\AppData\Local\Temp\1\ipykernel_9772\1225439327.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('movielens/users.dat',sep='::', header=None, names=unames)
C:\Users\arunch\AppData\Local\Temp\1\ipykernel_9772\1225439327.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table('movielens/ratings.dat',
C:\Users\arunch\AppData\Local\Temp\1\ipykernel_9772\1225439327.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); yo

In [3]:
movielens = pd.merge(pd.merge(ratings, users), movies)

In [4]:
movielens.gender.value_counts()

M    753769
F    246440
Name: gender, dtype: int64

In [5]:
movie_gender = movielens.pivot_table(values='rating', index='title', columns='gender', aggfunc=['mean', 'size'])
movie_gender['totalratings'] = movie_gender[('size', 'F')] + movie_gender[('size', 'M')]

In [6]:
sorted_ratingcnt = movie_gender.sort_values(by='totalratings', ascending=False)
sorted_ratingcnt[:5]

mean             size  \
gender                                                     F         M      F   
title                                                                           
American Beauty (1999)                              4.238901  4.347301  946.0   
Star Wars: Episode IV - A New Hope (1977)           4.302937  4.495307  647.0   
Star Wars: Episode V - The Empire Strikes Back ...  4.106481  4.344577  648.0   
Star Wars: Episode VI - Return of the Jedi (1983)   3.865237  4.069058  653.0   
Jurassic Park (1993)                                3.579407  3.814197  573.0   

                                                           totalratings  
gender                                                   M               
title                                                                    
American Beauty (1999)                              2482.0       3428.0  
Star Wars: Episode IV - A New Hope (1977)           2344.0       2991.0  
Star Wars: Episode V - The Empire Strikes Back ...  2342.0       2990.0  
Star Wars: Episode VI - Return of the Jedi (1983)   2230.0       2883.0  
Jurassic Park (1993)                                2099.0       2672.0

In [7]:
active_ratings = movie_gender[movie_gender.totalratings >= 250]
active_ratings['ratingdiff'] = active_ratings[('mean', 'M')] - active_ratings[('mean', 'F')]

C:\Users\arunch\AppData\Local\Temp\1\ipykernel_9772\3977236505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_ratings['ratingdiff'] = active_ratings[('mean', 'M')] - active_ratings[('mean', 'F')]


In [8]:
moviestats = movielens.groupby(by='title').agg({'rating': ['size', 'mean', 'std']})
active_movies = moviestats[moviestats[('rating', 'size')] >= 250]

In [9]:
# most disagreement
active_movies.sort_values(by=('rating', 'std'), ascending=False)[:5]

rating                    
                                        size      mean       std
title                                                           
Dumb & Dumber (1994)                     660  3.192424  1.321333
Blair Witch Project, The (1999)         1237  3.031528  1.316368
Natural Born Killers (1994)              700  3.144286  1.307198
Tank Girl (1995)                         358  2.614525  1.277695
Rocky Horror Picture Show, The (1975)   1233  3.291160  1.260177

In [10]:
# Higly rated movies by women
sorted_fratingcnt = active_ratings.sort_values(by=('mean', 'F'), ascending=False)
sorted_fratingcnt[:5]

mean             size  \
gender                                                     F         M      F   
title                                                                           
Close Shave, A (1995)                               4.644444  4.473795  180.0   
Wrong Trousers, The (1993)                          4.588235  4.478261  238.0   
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)       4.572650  4.464589  117.0   
Wallace & Gromit: The Best of Aardman Animation...  4.563107  4.385075  103.0   
Schindler's List (1993)                             4.562602  4.491415  615.0   

                                                           totalratings  \
gender                                                   M                
title                                                                     
Close Shave, A (1995)                                477.0        657.0   
Wrong Trousers, The (1993)                           644.0        882.0   
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)        353.0        470.0   
Wallace & Gromit: The Best of Aardman Animation...   335.0        438.0   
Schindler's List (1993)                             1689.0       2304.0   

                                                   ratingdiff  
gender                                                         
title                                                          
Close Shave, A (1995)                               -0.170650  
Wrong Trousers, The (1993)                          -0.109974  
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)       -0.108060  
Wallace & Gromit: The Best of Aardman Animation...  -0.178032  
Schindler's List (1993)                             -0.071187

In [11]:
# rated higly by women and lowly by men
sorted_ratingdiff = active_ratings.sort_values(by='ratingdiff')
sorted_ratingdiff[:5]

mean             size        totalratings  \
gender                            F         M      F      M                
title                                                                      
Dirty Dancing (1987)       3.790378  2.959596  291.0  396.0        687.0   
Jumpin' Jack Flash (1986)  3.254717  2.578358  106.0  268.0        374.0   
Grease (1978)              3.975265  3.367041  283.0  534.0        817.0   
Little Women (1994)        3.870588  3.321739  170.0  115.0        285.0   
Steel Magnolias (1989)     3.901734  3.365957  173.0  235.0        408.0   

                          ratingdiff  
gender                                
title                                 
Dirty Dancing (1987)       -0.830782  
Jumpin' Jack Flash (1986)  -0.676359  
Grease (1978)              -0.608224  
Little Women (1994)        -0.548849  
Steel Magnolias (1989)     -0.535777

In [12]:
# rated highly by men and lowly by women
sorted_ratingdiff[::-1][:5]

mean             size         \
gender                                         F         M      F      M   
title                                                                      
Good, The Bad and The Ugly, The (1966)  3.494949  4.221300   99.0  723.0   
Kentucky Fried Movie, The (1977)        2.878788  3.555147   33.0  272.0   
Dumb & Dumber (1994)                    2.697987  3.336595  149.0  511.0   
Longest Day, The (1962)                 3.411765  4.031447   34.0  318.0   
Cable Guy, The (1996)                   2.250000  2.863787   84.0  301.0   

                                       totalratings ratingdiff  
gender                                                          
title                                                           
Good, The Bad and The Ugly, The (1966)        822.0   0.726351  
Kentucky Fried Movie, The (1977)              305.0   0.676359  
Dumb & Dumber (1994)                          660.0   0.638608  
Longest Day, The (1962)                       352.0   0.619682  
Cable Guy, The (1996)                         385.0   0.613787

In [13]:
movielens[:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [14]:
print(movielens.shape)
# print(movielens.user_id.unique().shape)
print(movielens.user_id.astype('object').describe())
print(movielens.movie_id.astype('object').describe())

(1000209, 10)
count     1000209
unique       6040
top          4169
freq         2314
Name: user_id, dtype: int64
count     1000209
unique       3706
top          2858
freq         3428
Name: movie_id, dtype: int64


In [15]:
# Train and Test sets

# Use entire movielens set for generating train and test set (so that we have more training data for similarity scores)
sample = movielens.copy(deep=True)
# sample = movielens.loc[np.random.choice(movielens.index, size=10000, replace=False)]
# Take userids that have atleast 3 ratings
userid_min_ratingcnt = sample.user_id.value_counts() > 3
sample = sample[userid_min_ratingcnt[sample.user_id].values]

In [16]:
print(sample.shape)
# print(movielens.user_id.unique().shape)
print(sample.user_id.astype('object').describe())
print(sample.movie_id.astype('object').describe())

(1000209, 10)
count     1000209
unique       6040
top          4169
freq         2314
Name: user_id, dtype: int64
count     1000209
unique       3706
top          2858
freq         3428
Name: movie_id, dtype: int64


In [17]:
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df

In [18]:
sample['for_testing'] = False
grouped = sample.groupby('user_id', group_keys=False).apply(assign_to_set)
train = sample[grouped.for_testing == False]
test = sample[grouped.for_testing == True]

In [19]:
print(train.shape)
# print(movielens.user_id.unique().shape)
print(train.user_id.astype('object').describe())
print(train.movie_id.astype('object').describe())

(797758, 11)
count     797758
unique      6040
top         4169
freq        1851
Name: user_id, dtype: int64
count     797758
unique      3673
top         2858
freq        2748
Name: movie_id, dtype: int64


In [20]:
test.shape

(202451, 11)

In [21]:
# Evaluation

def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))


def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    ids_to_estimate = zip(test.user_id, test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    return compute_rmse(estimated, real)

In [22]:
def predict(t, estimate_f):
    test_predictions = t.copy(deep=True)
    test_predictions['prediction'] = test_predictions.apply(lambda x: estimate_f(x.user_id, x.movie_id), axis=1)

    return test_predictions

In [23]:
def evaluate_predictions(t):
    estimated = t.prediction.values
    real = t.rating.values
    return compute_rmse(estimated, real)

In [24]:
ratings_pivot = train.pivot_table(columns='user_id', index='movie_id', values='rating')
sim_matrix = ratings_pivot.corr()
sim_matrix_abs = sim_matrix.abs()
cov_matrix = ratings_pivot.cov()

u_mean_rating = ratings_pivot.mean()
u_var_rating = ratings_pivot.var()

var_not_zero = u_var_rating.loc[u_var_rating != 0]
m = u_mean_rating[var_not_zero.index]

norm_differential = pd.DataFrame(0, index=ratings_pivot.index, columns=ratings_pivot.columns)

# (x - xmean)/varx
norm_differential[var_not_zero.index] = ratings_pivot[var_not_zero.index].apply(lambda x: (x - m)/var_not_zero, axis=1)

In [37]:
user_meanrating = train.groupby(by='user_id').agg({'rating': ['mean', 'size']})
item_meanrating = train.groupby(by='movie_id').agg({'rating': ['mean', 'size']})

In [38]:
# take mean of ratings for each user
def contentbased_simplemean(u, i):
        return user_meanrating.loc[u, ('rating', 'mean')]

# take mean of ratings for each movie
def collab_simplemean(u, i):
        if i not in item_meanrating.index:
                return 3.0
        return item_meanrating.loc[i, ('rating', 'mean')]

In [39]:
user_meanrating[:5]

rating     
             mean size
user_id               
1        4.238095   42
2        3.776699  103
3        3.875000   40
4        4.375000   16
5        3.227848  158

In [26]:
evaluate(contentbased_simplemean)

1.0346313914136418

In [27]:
evaluate(collab_simplemean)

0.9782957472499411

In [28]:
min_rating = train.rating.min()
max_rating = train.rating.max()

In [29]:
# random rating scheme. using random int (in range of ratings) using uniform dist
def rand_intrating(u, i):
        return random.randint(min_rating, max_rating)

In [30]:
evaluate(rand_intrating)

1.893609398328394

In [31]:
# random rating scheme. random float with uniform dist
def rand_float_uniform(u, i):
        return random.uniform(min_rating, max_rating)

In [32]:
evaluate(rand_float_uniform)

1.7069712755035262

In [33]:
# normal float dist around mid point
def rand_float_normal(u, i):
        return random.normalvariate(3, 0.5)

In [34]:
evaluate(rand_float_normal)

1.3550013597941573

In [35]:
def const(u, i):
    return 3


In [36]:
evaluate(const)

1.2586836744476766

In [37]:
train[:2]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,False
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama,False


In [35]:
sim_matrix_check = (~sim_matrix_abs.isna()) & (sim_matrix_abs != 1)
sim_matrix_check.any(axis=None)

True

In [25]:
print(ratings_pivot.shape)
print(ratings_pivot.dropna(how='all').dropna(how='all', axis=1).shape)

(3673, 6040)
(3673, 6040)


In [36]:
print(sim_matrix_abs.shape)
print(sim_matrix_abs.dropna(how='all').dropna(how='all', axis=1).shape)
print(sim_matrix_check.shape)
print(cov_matrix.shape)
print(cov_matrix.dropna(how='all').dropna(how='all', axis=1).shape)

(6040, 6040)
(6037, 6037)
(6040, 6040)
(6040, 6040)
(6040, 6040)


In [26]:
(u_var_rating == 0).describe()

count      6040
unique        1
top       False
freq       6040
dtype: object

In [27]:
ratings_pivot[:2]

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
norm_differential[:2]

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.887029,NaN,NaN,NaN,NaN,NaN,NaN,0.147228,NaN,1.203674,...,NaN,-0.084776,NaN,NaN,0.595009,NaN,NaN,NaN,NaN,-0.448051
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.203674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
cov_matrix[:2]


user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.416376,0.45000,-0.666667,0.50,-0.500000,4.666667e-01,0.000000,-0.200000,0.035714,0.054348,...,1.776357e-16,0.666667,0.500,NaN,0.200000,-0.098039,-0.261905,NaN,0.054545,-0.208333
2,0.450000,0.98439,0.267857,-0.25,-0.288462,8.881784e-17,-0.063636,-0.077206,0.678571,-0.053977,...,-3.214286e-01,0.500000,0.125,NaN,0.821053,0.242525,0.105263,0.0,0.200000,0.048295


In [35]:
def pred_rating(user, movie):
    if not(movie in ratings_pivot.index):
        return 3.0

    movie_vec = ratings_pivot.loc[movie]
    movie_vec = movie_vec.dropna()

    # get sim vec conditioned on valid (not na) movie_vec values
    sim_vec = sim_matrix_abs[user].loc[movie_vec.index]
    weighted_rating = (movie_vec * sim_vec).dropna()

    # if no weighted ratings (no overlapping valid movie vec and sim vec values), use movie_vec to compute rating
    if weighted_rating.size <= 0:
        return movie_vec.mean() if movie_vec.size > 0 else 3.0

    sim_vec_sum = sim_vec.sum()

    # if sim space for this context evaluates to 0, use movie_vec to compute rating 
    if sim_vec_sum <= 0:
        return movie_vec.mean() if movie_vec.size > 0 else 3.0
    
    # use similarity as confidence to weight the ratings of other users
    return weighted_rating.sum()/sim_vec_sum

In [30]:
def pred_rating_norm(user, movie):
    if not(movie in norm_differential.index):
        return 3.0

    # get norm differential (from mean) for this movie, for each user
    norm_diff_movie_vec = norm_differential.loc[movie]

    # get covariance of this user wrt each user
    cov_vec = cov_matrix[user]

    # get other users' ratings expressed as differential from mean of this user
    diff_rating_user = norm_diff_movie_vec * cov_vec

    # add mean of this user to get other users' ratings expressed in terms of this user's scale
    rating_user = diff_rating_user + u_mean_rating[user]

    movie_vec = rating_user.dropna()

    # get sim vec conditioned on valid (not na) movie_vec values. This is so that we only consider sim values, where movie ratings are defined to calculate sim_vec_sum
    sim_vec = sim_matrix_abs[user].loc[movie_vec.index]
    
    weighted_rating = (movie_vec * sim_vec).dropna()

    # if no weighted ratings (no overlapping valid movie vec and sim vec values), use movie_vec to compute rating
    if weighted_rating.size <= 0:
        return movie_vec.mean() if movie_vec.size > 0 else 3.0

    sim_vec_sum = sim_vec.sum()

    # if sim space for this context evaluates to 0, use movie_vec to compute rating 
    if sim_vec_sum <= 0:
        return movie_vec.mean() if movie_vec.size > 0 else 3.0
    
    # use similarity as confidence to weight the ratings of other users
    return weighted_rating.sum()/sim_vec_sum

In [31]:
p = predict(test, pred_rating_norm)

In [32]:
p.to_csv('movielens/predictions_norm.csv', index=None)

In [36]:
p_nonorm = predict(test, pred_rating)
# p.to_csv('movielens/predictions.csv', index=None)

In [38]:
evaluate_predictions(p)

0.9700281592662616

In [37]:
evaluate_predictions(p_nonorm)

0.9695074115603279

In [27]:
prediction_undefined = p[p.prediction.isna()]
prediction_undefined

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing,prediction


In [33]:
pred_rating(3765, 2258)

2.0000000000000004

In [71]:
evaluate(pred_rating)

0.9691078861431314

In [34]:
predict_3 = p[p.prediction == 3.0]
print(predict_3.shape)
predict_2 = p[p.prediction == 2.0]
print(predict_2.shape)
predict_4 = p[p.prediction == 4.0]
print(predict_4.shape)
predict_1 = p[p.prediction == 1.0]
print(predict_1.shape)
predict_5 = p[p.prediction == 5.0]
print(predict_5.shape)
predict_0 = p[p.prediction == 0]
print(predict_0.shape)

(42, 12)
(9, 12)
(17, 12)
(19, 12)
(4, 12)
(0, 12)
